# Clustering point cloud data from the Imaging Radar

This notebook is intended to cluster the point cloud data (PCD) from the imaging radar using the K-Means clustering algorithm. The PCD is received from the data acquisition team in the .csv format.


### Importing all the necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math as m
%matplotlib qt
from sklearn.cluster import KMeans

### Importing the PCD of Radar

Download the .csv file from the shared OneDrive folder and save it in the same folder as the Git repository. The csv file is loaded in the next code block.

In [2]:
data = pd.read_csv("3D_point_all_frames_Test_220315_TR13_Lidar_comparison_play_1.csv")
data.head()

data_rad = data[['x','y','z']].values

data_rad[:,2] = data_rad[:,2]*-1                      # Reversing orientation of Z

print(data_rad.shape)



(2094, 3)


### Cleaning Radar data

In [3]:
k = []

H = -0.5                             #Height of the sensor from the ground
max_y = 10                            # Distance of 7 people from the radar
max_z = 1.5                          # Height of 7 people (0.5+1.5m = 2m from the ground)
for i in range(len(data_rad[:,2])):
    if data_rad[i,2]>H and data_rad[i,1]<max_y and data_rad[i,2]<max_z and data_rad[i,0]<4:
        k.append(data_rad[i])
        
data_rad=np.array(k)


print(data_rad.shape)

(1469, 3)


### Importing Lidar Data

In [4]:
data_lid = pd.read_csv("cloud_2.csv")
data_lid = data_lid[['x','y','z']].values
print(data_lid.shape)

(22322, 3)


### Cleaning Lidar Data

In [5]:
lidar = []


# Lidar data is scaled to focus on the 7 objects
max_x = 20 
max_y = 7.5
max_z = 5


for i in range(len(data_lid[:,2])):
    if data_lid[i,0]<max_x and data_lid[i,1]<max_y and data_lid[i,2]<max_z and data_lid[i,0]>-max_x and data_lid[i,1]>-max_y and data_lid[i,2]>-max_z:
        lidar.append(data_lid[i])
        
        
data_lid=np.array(lidar)

print(data_lid.shape)

(17768, 3)


### Estimation of optimal 'K' value using the Elbow rule for radar data

The elbow curve is visualised. The optimal value of K is at the point when the curve starts to straighten out. 

In [6]:
WCSS = []

for i  in range(1,15):
    kmeans_model = KMeans(n_clusters = i, init = 'k-means++')
    kmeans_model.fit(data_rad)
    WCSS.append(kmeans_model.inertia_)
    
    
fig = plt.figure(figsize = (7,7))
plt.plot(range(1,15),WCSS, linewidth=4, markersize=12,marker='o',color = 'red')
plt.xticks(np.arange(15))
plt.xlabel("Number of Clusters")
plt.ylabel("WCSS")
plt.show()

C:\Users\20181051\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:881: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=6.
  warnings.warn(


## K-Means clustering

The number of clusters is initialized and the algorithm is defined. The number of points in each cluster are plotted.

In [7]:
kmeans_model = KMeans(n_clusters=9, init = "k-means++", max_iter = 300, n_init = 10, random_state = 0).fit(data_rad)
radar_clusters = kmeans_model.fit_predict(data_rad)


#check number of clusters and number of points in each cluster
plt.figure(2)
sns.countplot(radar_clusters)
plt.show()

C:\Users\20181051\Anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


### Orientation of Radar data to Lidar

To align the axes of the radar and the lidar, the radar data is rotated by a 90 degrees wrt to the z-axis (xy-plane). This means the x-axis of the radar will be transformed to be aligned with the y-axis of the radar. The new axis for radar data has x and y axes interchanged. The distance between the radar and lidar was x = 0.10m and y= 0.25m. The radar data is also transformed 1.5 degrees to correct tilt.


In [8]:
rot_val = 90*(m.pi/180)


trans_mat=([m.cos(rot_val),-m.sin(rot_val), 0],
          [m.sin(rot_val), m.cos(rot_val), 0],
          [0,              0,              1])
data_rad=np.matmul(data_rad, trans_mat)

print(data_rad.shape)
data_rad[:,1]=data_rad[:,1]+0.25
data_rad[:,0]=data_rad[:,0]-0.10


rot_val_z = 1.5*(m.pi/180)


trans_mat_z=([1,0, 0],
          [0, m.cos(rot_val_z), -m.sin(rot_val_z)],
          [0, m.sin(rot_val_z), m.cos(rot_val_z)])
data_rad=np.matmul(data_rad, trans_mat_z)


(1469, 3)


### Visualize the data

The clustered radar data is visualized along with the lidar data.

In [9]:
fig = plt.figure(figsize = (15,15))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data_rad[radar_clusters == 0,0],data_rad[radar_clusters == 0,1],data_rad[radar_clusters == 0,2], s = 5 , color = 'orange', label = "Jose")
ax.scatter(data_rad[radar_clusters == 1,0],data_rad[radar_clusters == 1,1],data_rad[radar_clusters == 1,2], s = 5 , color = 'red', label = "Lijo")
ax.scatter(data_rad[radar_clusters == 2,0],data_rad[radar_clusters == 2,1],data_rad[radar_clusters == 2,2], s = 5 , color = 'green', label = "Raj")
ax.scatter(data_rad[radar_clusters == 3,0],data_rad[radar_clusters == 3,1],data_rad[radar_clusters == 3,2], s = 5 , color = 'orange', label = "Lijo")
ax.scatter(data_rad[radar_clusters == 4,0],data_rad[radar_clusters == 4,1],data_rad[radar_clusters == 4,2], s = 5 , color = 'cyan', label = "Prabhat")
ax.scatter(data_rad[radar_clusters == 5,0],data_rad[radar_clusters == 5,1],data_rad[radar_clusters == 5,2], s = 5 , color = 'purple', label = "Prabhat")
ax.scatter(data_rad[radar_clusters == 6,0],data_rad[radar_clusters == 6,1],data_rad[radar_clusters == 6,2], s = 5 , color = 'pink', label = "Henry")
ax.scatter(data_rad[radar_clusters == 7,0],data_rad[radar_clusters == 7,1],data_rad[radar_clusters == 7,2], s = 5 , color = 'purple',label = "Federico")
ax.scatter(data_rad[radar_clusters == 8,0],data_rad[radar_clusters == 8,1],data_rad[radar_clusters == 8,2], s = 5 , color = 'blue', label = "Ani")
ax.scatter(data_lid[:,0],data_lid[:,1],data_lid[:,2], s = 1 , color = 'grey', label = "Lidar Data")
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_zlabel('z')
ax.legend()
plt.show()